<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/faiss/faiss_index_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (213 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [1]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [2]:
import tarfile

tar = tarfile.open('sift.tar.gz', 'r:gz')
tar.extractall()

In [3]:
import numpy as np

def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d+1)[:, 1:].copy().view('float32')

In [4]:
# 검색할 데이터
xb = read_fvecs('./sift/sift_base.fvecs') #1M samples
# some query vectors
xq = read_fvecs('./sift/sift_query.fvecs')
# 하나의 쿼리를 추출
xq = xq[0].reshape(1, xq.shape[1])

In [5]:
xq.shape

(1, 128)

In [6]:
xb.shape

(1000000, 128)

In [7]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

# IndexFlatL2
- balance of search spped of search quality
- 대부분 검색 품질이 높으면 검색 속도가 낮아진다.
- 10억개의 벡터가 있고 1분에 100개의 쿼리를 수행한다고 하면 오랜시간이 걸릴 것이다.
- 그리고 이걸 실행하려면 미친 하드웨어가 필요하므로 완전 검색에서는 빠른 색인을 사용할 수 없지만 사용 방법을 보여드리겠습니다~

In [16]:
d = 128
k = 10 # how many result do i want

import faiss

index = faiss.IndexFlatIP(d) # L2보다 약간 빠름. 실제로는 거의 차이가 없음
index.add(xb)


In [17]:
%%time
D, I = index.search(xq, k) # 샘플 갯수와 쿼리 벡터

CPU times: user 223 ms, sys: 0 ns, total: 223 ms
Wall time: 208 ms


In [18]:
I # 10개의 가장 가까운 인덱스

array([[932085, 934876, 561813, 708177, 706771, 695756, 435345, 701258,
        872728, 455537]])

In [19]:
baseline = I[0].tolist()
baseline

[932085,
 934876,
 561813,
 708177,
 706771,
 695756,
 435345,
 701258,
 872728,
 455537]

# LSH (Locality Sensitive Hashing)
- 50 : 50
- hashing 기능 : hash buckets에 값이 중복되는 충돌을 최소화
- python의 dictionary 같은 구조
- 가까운 버킷을 찾은다음 검색 범위를 제한한다.
- 모든 곳을 검색할 필요가 없음.
- 아주 직관적

In [23]:
nbits = d*5 # 데이터의 차원에 따라 확장됨

index = faiss.IndexLSH(d,nbits)
index.add(xb)

In [24]:
%%time

D, I = index.search(xq, k)

CPU times: user 120 ms, sys: 565 µs, total: 120 ms
Wall time: 70.5 ms


In [25]:
np.in1d(baseline, I)

array([False,  True,  True, False, False,  True,  True, False,  True,
       False])

- LSH 알고리즘을 통해 얻은 10개의 결과값.
- 대부분이 일치함.
- 합리적으로 좋은 회수율.
- 시간이 빨라짐.
- nbits를 조정하여 더 좋은 결괏값을 얻을 수 있음.
- recall graph를 보면 차원을 증가시키면 좋은 recall을 얻을 수 있지만, 차원의 저주가 있다.
- 낮은 차원일 수록 좋다.

# HNSW(Hierarchical Navigable Small World)
- small world graph를 탐색.
- hops로 접근
- 인덱스 추가에 더 오래 걸림(더 정확하게 하기 위해)
- ef_construction은 크게 해도될것같고.. (대신 메모리 소비 많음)
- ef_search는 잘 조절

In [32]:
M = 16 # 50~16 number of 꼭짓점의 연결
ef_search = 16 #검색의 깊이 (네트워크 더 많이 검색하려는 경우 high or low)
ef_construction = 64 #네트워크 구성할 때 네트워크의 양. 얼마나 자세하게 네트워크를 구성할 것인가. 검색 시간에는 영향을 주지 않으므로 높게 설정

In [33]:
index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efSearch = ef_search
index.hnsw.efConstruction = ef_construction

index.add(xb)

In [34]:
%%time

D, I = index.search(xq, k)

CPU times: user 3.24 ms, sys: 0 ns, total: 3.24 ms
Wall time: 3.02 ms


In [35]:
np.in1d(baseline, I)

array([ True,  True, False,  True,  True,  True, False, False, False,
       False])

# IVF(Inverted File Index)
- Super cool index
- Clustering datapoint를 기반으로 함.
- 보로노이 셀 or 디리클레 테셀레이션
- 각 영역의 중심과의 거리를 계산
- 가까운 곳의 영역 안의 벡터와만 비교
### Edge Problem
- 쿼리를 사용하면 셀 중 하나의 가장자리에 있다. 그리고 n개의 probe value를 정하면, 1로 되어있다면 검색하는 영역의 수. 
- 가까운 중심 노드가 있어도 검색하지 않을 것이다. 엣지로 영역이 막혀 있으니. 따라서 search scope를 늘림으로써 (nprobe의 증가) 여러개의 셀을 검색하므로 해결할 수 있다.

In [36]:
nlist = 128 # 데이터 내에서 가질 중심의 수
quantizer = faiss.IndexFlatIP(d) # 최종 검색을 수행하는 방법

index = faiss.IndexIVFFlat(quantizer, d, nlist) 
index.is_trained 

False

In [39]:
index.train(xb) # 다른 클러스터링은 훈련이나 최적화가 필요없지만, 사용하기 전 인덱스 트레인을 작성하고 모든 벡터를 전달하며ㅡ 속도는 느리지 않다.

In [40]:
index.add(xb)

In [44]:
index.nprobe = 2 # 2로 증가했더니 더 높은 성능

In [45]:
%%time
D, I = index.search(xq, k) # Low quality 결과를 제외하고 가장 빠른 것 같음.

CPU times: user 17.7 ms, sys: 0 ns, total: 17.7 ms
Wall time: 20.3 ms


In [46]:
np.in1d(baseline, I)

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True])